# Generate Lowest Price Data
This notebook generates the lowest price data from the renfe dataset.
This is done in a separate file, because it is a lengthy computation, so we will save it in a csv file.

In [1]:
import pandas as pd
import string
import os

# This next import is some date functions I created
import timefuns as tf

Before importing the data, we get the column names, so we can remove the unnamed column.

In [2]:
cols = list(pd.read_csv("renfe.csv", nrows =1))
print(cols)

['Unnamed: 0', 'insert_date', 'origin', 'destination', 'start_date', 'end_date', 'train_type', 'price', 'train_class', 'fare']


In [3]:
raw_data = pd.read_csv("renfe.csv", dtype = 'str', usecols = [i for i in cols if i != 'Unnamed: 0'])

We then restrict the data to trains from Madrid to Barcelona, train type AVE, and drop empty price entries.

In [4]:
stage = raw_data.loc[(raw_data['origin'] == 'MADRID') & 
                     (raw_data['destination'] == 'BARCELONA') &
                     (raw_data['train_type'] == 'AVE')]
stage = stage.dropna(subset = ['price'])

We then restrict to entries of fare type 'Promo' and train class 'Turista'

In [5]:
stageProm = stage.loc[stage['fare'] == 'Promo']
stageFlex = stage.loc[stage['fare'] == 'Flexible']

In [6]:
stageTur = stageProm.loc[stageProm['train_class'] == 'Turista']

We create a function that returns the lowest price within a one-hour window of a given time

In [7]:
def getLowestPrice(date):
    return stageTur.loc[(stageTur['start_date'].apply(tf.getDate) == tf.getDate(date)) &
                        (stageTur['start_date'].apply(tf.getHourFloat) - tf.getHourFloat(date) < 1.1) &
                        (stageTur['start_date'].apply(tf.getHourFloat) - tf.getHourFloat(date) > -1.1)]\
                       ['price'].astype(float).min()

We create a data set with all the unique start times, lowest prices and weekdays

In [8]:
stageDatesTur = pd.DataFrame(stageTur['start_date'].unique())
stageDatesTur.columns = ['start_date']
stageDatesTur['lowest_price'] = stageDatesTur['start_date'].apply(getLowestPrice)
stageDatesTur['weekday'] = stageDatesTur['start_date'].apply(tf.weekday)

We save the dataset into a csv file

In [9]:
stageDatesTur.to_csv(r'.\renfe_lowest_price.csv', index = None)

In [10]:
stageDatesTur.head()

,start_date,lowest_price,weekday
0,2019-04-27 07:30:00,49.55,saturday
1,2019-04-27 16:30:00,58.15,saturday
2,2019-04-27 17:30:00,58.15,saturday
3,2019-04-27 20:30:00,58.15,saturday
4,2019-04-28 06:20:00,49.55,sunday
